<a href="https://colab.research.google.com/github/pandemic-tracking/viz-gen/blob/main/ptc_pivottable_hexmaps_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
!pip install --upgrade gspread

In [ ]:
!wget "https://github.com/donmeltz/US-States---Hexbins/blob/master/Shapefile/Web%20Mercator%20(EPSG%203857).zip?raw=true" -O hexbin.zip

In [ ]:
!unzip hexbin.zip

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import pytz

import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

from pathlib import Path

pd.set_option("display.precision", 4)

now_est = datetime.now().astimezone(pytz.timezone("US/Eastern"))

now_est_time = now_est.strftime("%Y-%m-%d, %H:%M:%S ET")
now_est_date = now_est.strftime("%Y-%m-%d")
now_est_timestamp = now_est.strftime("%Y%m%d_%H%M%S")
now_utc_timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
print(now_est_time, now_est_date, now_est_timestamp, now_utc_timestamp)

In [ ]:
# adapting from https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602

def ptc_theme():
    axisColor = "#808080"
    gridColor = "#DEDDDD"
    markColor = "#000000"
    font = 'Arial'
    labelFont = 'Arial'
  
    return {
          "config": {
              "title": {
                  "fontSize": 16,
                  "font": font,
                  "anchor": "start", # equivalent of left-aligned.
                  "fontColor": "#000000",
                  "fontWeight": "normal",
              },
              "text": {
                  "font": font,
                  "labelFont": labelFont,
              },
              "header": {
                  "font": font,
                  "labelFont": labelFont,
                  "titleFont": font,
              },
              "axisX": {
                  "domain": False,
                  "domainColor": axisColor,
                  "labelColor": axisColor,
                  "domainWidth": 1,
                  "grid": False,
                  "labelFont": labelFont,
                  "labelFontSize": 12,
                  "labelAngle": 0, 
                  "tickColor": axisColor,
                  "tickSize": 5, # default, including it just to show you can change it
                  "titleFont": font,
                  "titleFontSize": 12,
                  "titlePadding": 10, # guessing, not specified in styleguide
                  "title": "X Axis Title (units)", 
              },
              "axisY": {
                  "domain": False,
                  "grid": True,
                  "gridColor": gridColor,
                  "gridWidth": 1,
                  "labelFont": labelFont,
                  "labelColor": axisColor,
                  "labelFontSize": 12,
                  "labelAngle": 0,
                  "labelAnchor": "end",
                  "labelAlign": "right",  
                  "ticks": False, # even if you don't have a "domain" you need to turn these off.
                  "titleFont": font,
                  "titleFontSize": 12,
                  "titlePadding": 10, # guessing, not specified in styleguide
                  "title": "Y Axis Title (units)", 
                  # titles are by default vertical left of axis so we need to hack this 
                  "titleAngle": 0, # horizontal
                  "titleY": -10, # move it up
                  "titleX": 18, # move it to the right so it aligns with the labels 
              },
            #   "range": {
            #       "category": main_palette,
            #       "diverging": sequential_palette,
            #   },
              "legend": {
                  "labelFont": labelFont,
                  "labelFontSize": 12,
                  "symbolType": "circle", # just 'cause
                  "symbolSize": 100, # default
                  "titleFont": font,
                  "titleFontSize": 12,
                  "title": "", # set it to no-title by default
                  "orient": "right", # so it's right next to the y-axis
                  "offset": 0, # literally right next to the y-axis.
              },
              "view": {
                  "stroke": "transparent", # altair uses gridlines to box the area where the data is visualized. This takes that off.
              },
        }
    }

alt.themes.register("my_custom_theme", ptc_theme)
alt.themes.enable("my_custom_theme")

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('/content/Web Mercator (EPSG 3857)/US_HexBinStates_EPSG3857.shp')
gdf.loc[:, 'lon'] = gdf.geometry.centroid.x
gdf.loc[:, 'lat'] = gdf.geometry.centroid.y
gdf.head(2)

In [ ]:
data_df = pd.read_csv('https://github.com/pandemic-tracking/bi/raw/main/US%20states%20breakthrough%20reporting%20-%20Snapshot.csv')
collection_date = '2021-11-04'

In [ ]:
data_df

In [ ]:
list(data_df.columns)

In [ ]:
metrics_cols = [
  'BI cases',
  'BI cases as a percent of fully vaccinated',
  'BI cases as a percent of total cases',
  'BI cases per 100k fully vaccinated',
  'BI hosp',
  'BI hosp as a percent of BI cases',
  'BI hosp as a percent of fully vaccinated',
  'BI hosp as a percent of total hosp',
  'BI hosp due to COVID',
  'BI hosp due to COVID as a percent of fully vaccinated',
  'BI hosp due to COVID per 100k fully vaccinated',
  'BI hosp per 100k fully vaccinated',
  'BI deaths',
  'BI deaths as a percent of BI cases',
  'BI deaths as a percent of fully vaccinated',
  'BI deaths as a percent of total deaths',
  'BI deaths due to COVID',
  'BI deaths due to COVID as a percent of BI cases',
  'BI deaths due to COVID as a percent of fully vaccinated',
  'BI deaths due to COVID as a percent of total deaths due to COVID',
  'BI deaths due to COVID per 100k fully vaccinated',
  'BI deaths per 100k fully vaccinated',
  'BI variant cases',
  'not BI cases',
  'not BI cases as a percent of not vaccinated',
  'not BI cases as a percent of total cases',
  'not BI cases per 100k not vaccinated',
  'not BI hosp',
  'not BI hosp as a percent of not vaccinated',
  'not BI hosp as a percent of total hosp',
  'not BI hosp due to COVID as a percent of total hosp due to COVID',
  'not BI hosp due to COVID per 100k not vaccinated',
  'not BI hosp per 100k not vaccinated',
  'not BI deaths',
  'not BI deaths as a percent of total deaths',
  'not BI deaths as percent of not vaccinated',
  'not BI deaths due to COVID',
  'not BI deaths due to COVID as a percent of total deaths due to COVID',
  'not BI deaths due to COVID as percent of not vaccinated',
  'not BI deaths due to COVID per 100k not vaccinated',
  'not BI deaths per 100k',
  'Total cases (since fully vax was option)',
  'Total hosp (since fully vax was option)',
  'Total hosp due to COVID (since fully vax was option)',
  'Total deaths (since fully vax was option)',
  'Total deaths due to COVID (since fully vax was option)',
  'Total Individuals fully vaccinated',
  'Total Individuals not fully vaccinated',
]

In [ ]:
df = data_df[['Abbr']+metrics_cols].rename({'Abbr':'State'}, axis=1)

In [ ]:
df

In [ ]:
df[metrics_cols] = df[metrics_cols].notnull().astype(int)

In [ ]:
df.loc[df.shape[0]]= ['Count'] + df[metrics_cols].sum(axis=0).to_list()

In [ ]:
df

In [ ]:
gdf = pd.merge(gdf, df, how='left', left_on='Code', right_on='State')

In [ ]:
gdf.drop(['OBJECTID','SHAPE_Leng','SHAPE_Area'], axis=1, inplace=True)

In [ ]:
# BI hosps OR BI hosps due to COVID and same for BI deaths, also make category names display-ready

gdf['Reporting Breakthrough Hospitalizations'] = np.where((gdf['BI hosp'] == 1) | (gdf['BI hosp due to COVID']==1),1,0)
gdf['Reporting Breakthrough Deaths'] = np.where((gdf['BI deaths'] == 1) | (gdf['BI deaths due to COVID']==1),1,0)

gdf.rename({'BI cases':'Reporting Breakthrough Cases'}, axis=1, inplace=True)

In [ ]:
gdf = gdf.fillna(0)

In [ ]:
gdf[['Code','BI hosp','BI hosp due to COVID','Reporting Breakthrough Hospitalizations']]

In [ ]:
gdf

In [ ]:
gdf_melt = gdf.melt(id_vars=['Code','Name','FIPS','geometry','lon','lat','State'])

In [ ]:
gdf_melt

In [ ]:
cat_options = list(gdf_melt['variable'].unique())
cat_options

In [ ]:
# A dropdown filter
cat_dropdown = alt.binding_select(options=cat_options)
cat_select = alt.selection_single(fields=['variable'], bind=cat_dropdown, name="Metric", init={'variable':cat_options[0]})

base_map = alt.Chart(data=gdf_melt).mark_geoshape(stroke='black', strokeWidth=0.5).encode(
    color=alt.value('grey'),
    tooltip=["Code:O",'variable:O','value:Q'],
    ).project(
      type='identity', reflectY=True
).properties(width=550, height=300)

map = base_map.encode(
    color=alt.Color("value:Q", scale=alt.Scale(scheme='greens'), legend=None),
)

map_text = base_map.mark_text(size=12).encode(
    latitude='lat',
    longitude='lon',
    text='Code',
    color=alt.Color("value:Q", scale=alt.Scale(range=['#ffffff','#2b2b2b'], domain=[1,0]), legend=None),
    
)

display_text = base_map.mark_text(size=16, fontWeight=600, dx=0, dy=-140).encode(
    text='variable:O',
    color=alt.value('black')
).transform_filter(alt.datum.Code=='CA')

# hexmap = (base_map+map+map_text+display_text).transform_filter(cat_select
#                                             ).add_selection(cat_select
#                                             ).configure_view(strokeWidth=0)
# hexmap

In [ ]:
metric_count_dict = gdf[['Reporting Breakthrough Cases','Reporting Breakthrough Hospitalizations','Reporting Breakthrough Deaths']].sum().astype('int').to_dict()
metric_count_dict

In [ ]:
display_date = datetime.strptime(collection_date,"%Y-%m-%d").strftime("%B %-d %Y")
collection_date, display_date

In [ ]:
case_color, hosp_color, death_color, zero_color = '#3E85C6', '#0D5395', '#063763', '#dbdbdb'

def make_hexmap(metric, pos_color, neg_color):
  hexmap = (display_text.encode(
    color=alt.value(pos_color),
    text=alt.value(str(metric_count_dict[metric])+' '+metric)
   )+base_map
    +base_map.encode(color=alt.Color("value:Q", scale=alt.Scale(range=[pos_color, neg_color], domain=[1,0]), legend=None))
    +map_text
  ).transform_filter(alt.datum.variable==metric).resolve_scale(color='independent')
  return hexmap

case_map =  make_hexmap('Reporting Breakthrough Cases', case_color, zero_color)
hosp_map =  make_hexmap('Reporting Breakthrough Hospitalizations', hosp_color, zero_color)
death_map = make_hexmap('Reporting Breakthrough Deaths', death_color, zero_color)

tri_hexmaps = alt.vconcat(
    case_map|hosp_map,
    death_map,
    center=True,
).properties(title=alt.TitleParams(f'Breakthrough Data Reporting by State as of {display_date}', subtitle=['',''])
).configure_axis(
  labelFontSize=12,
  titleFontSize=16,
  labelColor='grey',
  # gridColor='#ccc',
  gridWidth=1,
  offset=0,
).configure_header(
  titleFontSize=16,
  labelFontSize=16, 
).configure_title(
  fontSize=16,
).configure_legend(
  titleFontSize=12,
  labelFontSize=12
).configure_view(strokeWidth=0)

tri_hexmaps

# Save to Drive

In [ ]:
# this is for saving altair charts to png and svg, based on https://colab.research.google.com/github/altair-viz/altair_saver/blob/master/AltairSaver.ipynb#scrollTo=ZiTDBCAM_Ni8
!pip install -q altair_saver
!npm install --silent vega-lite vega-cli canvas

In [ ]:
from pathlib import Path
from altair_saver import save

SAVE_PATH = Path('assets')
SAVE_PATH.mkdir(exist_ok=True)

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [ ]:
def assets_to_gdrive(folder_name, localdir_path = SAVE_PATH, parentdir_id='17Kx2uZbQv1r5U1M9x_OXS4lpMU5c6Ym8'):
  # search gdrive for snapshot folder and save assets there if it already exists. 
  folder_id = ''
  file_list = gdrive.ListFile({'q': f"'{parentdir_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed=false"}).GetList()
  for file1 in file_list:
      if file1['title'] == folder_name: 
        folder_id = file1['id']
        print(f'Found pre-existing gdrive folder named "{folder_name}" at',folder_id)
  # if not, create new folder
  if folder_id == '':
    folder = gdrive.CreateFile(metadata={'title': folder_name,
                                      'parents':[{'id': parentdir_id}],
                                      "mimeType": "application/vnd.google-apps.folder"
                                      })
    folder.Upload()
    folder_id = folder.get('id')
    print(f'Created new gdrive folder named "{folder_name}" at',folder_id)
  
  # upload all files within SAVE_PATH to snapshot folder
  for asset_file in localdir_path.iterdir():
    file_path = localdir_path/asset_file.name
    file1 = gdrive.CreateFile(metadata={'title':asset_file.name,
                                        'parents':[{'id': folder_id}],
                                        })
    file1.SetContentFile(file_path)
    file1.Upload()
    print('Saved file: ',asset_file.name)


In [ ]:
def save_vizassets(chart, save_path, filename, fmts=['html','json','png','svg','pdf']):
  for fmt in fmts:
    save(chart, f'{save_path}/{filename}.{fmt}')

In [ ]:
# put your stuff (i.e. dataframes, altair charts, input data files) to save here

In [ ]:
# examples:

# cpr_df.to_csv(SAVE_PATH/'cpr_df.csv')
# source_concat.to_csv(SAVE_PATH/'source_concat.csv')
# save_vizassets(state_vax_scatter, SAVE_PATH, f'state_vax_scatter_{now_utc_timestamp}')

gdf.to_csv(SAVE_PATH/'gdf.csv')
save_vizassets(hexmap, SAVE_PATH, f'hexmap_{now_utc_timestamp}')
save_vizassets(tri_hexmaps, SAVE_PATH, f'trihexmaps_{now_utc_timestamp}')

In [ ]:
# get the colab filename
from requests import get
nb_name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].replace('.ipynb','')
nb_id = get('http://172.28.0.2:9000/api/sessions').json()[0]['notebook']['path'].replace('fileId=','')

print(SAVE_PATH, nb_name, now_utc_timestamp, nb_id)

# create a snapshot of this currently running notebook and save to SAVE_PATH
downloaded_nb = gdrive.CreateFile({'id':nb_id})   # replace the id with id of file you want to access
downloaded_nb.GetContentFile(SAVE_PATH/f'{nb_name}_{now_utc_timestamp}.ipynb')

In [ ]:
# upload everything to gdrive
assets_to_gdrive(folder_name=f'{nb_name}_{now_utc_timestamp}')